In [81]:
import os
import pandas as pd
import json

os.chdir(r'T:\laupodteam\AIOS\Bram\language_modeling\MEDICAL_TEXT\RAW\NtvG')

In [3]:
nvtg = pd.read_feather("./ntvg_articles.feather")

In [71]:
# make function to pull out a list of dictionaries with the following keys:
# - body_clean
# - summary_clean
# - category
# - domain

# parameters:
# - stratified
# - fraction
# - random_state

def pull_dictionaries(df: pd.DataFrame, 
                      stratified: bool,
                      fraction: float, 
                      random_state: int, 
                      exclude_dict: dict=None):
    
    if isinstance(exclude_dict, dict):
        for k,v in exclude_dict.items():
            for _v in v:
                df = df.loc[df[k]!=_v]
    
    if stratified:
        # TODO: implement stratified sampling
        df = df.sample(frac=fraction, random_state=random_state)
    else:
        df = df.sample(frac=fraction, random_state=random_state)
    df = df.reset_index(drop=True)
    dictionaries = []
    for i in range(len(df)):
        summaries = df.loc[i, "summary_clean"]
        explanation = df.loc[i, "explanation_clean"]
        information_framework = df.loc[i, "information_framework_clean"]
        intro = df.loc[i, "intro_clean"]
        
        summary_string = "\r\n".join(summaries) if summaries is not None else None
        explanation_string = "\r\n".join(explanation) if explanation is not None else None
        information_framework_string = "\r\n".join(information_framework) if information_framework is not None else None
        intro_string = "\r\n".join(intro) if intro is not None else None
        
        txts = "\r\n".join(df.loc[i, "body_clean"])
        
        id = df.loc[i, "id"]        
        dictionary = {
                    "id": id,
                    "body_clean": txts,
                    "summary_clean": summary_string,
                    'explanation_clean': explanation_string,
                    'information_framework_clean': information_framework_string,
                    'intro_clean': intro_string,
                    "category": df.loc[i, "category"],
                    "domain": df.loc[i, "domain"]
                }
        dictionaries.append(dictionary)
    return dictionaries

In [79]:
dicts = pull_dictionaries(nvtg, stratified=True, fraction=0.05, random_state=42, exclude_dict={'category':['Redactioneel'], 'domain': []})

In [90]:
pd.DataFrame(dicts).to_json('./samples.jsonl', orient='records', lines=True, force_ascii=False)